# faiss 

In [1]:
#Install Packages
#!pip install faiss-cpu
#!pip install sentence-transformers

   ---------------------------------------- 14.6/14.6 MB 164.6 kB/s eta 0:00:00


In [2]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv(r"C:\Users\HP\Downloads\py\LangCHain Chatbots\sample_text.csv")
df.shape

(8, 2)

In [4]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


**Step 1 : Create source embeddings for the text column**



In [5]:
from sentence_transformers import SentenceTransformer

all-mpnet-base-v2 :This transformer model convert text into vector

In [6]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [9]:
vectors.shape # We have total 8 vectors

(8, 768)

In [10]:
dim = vectors.shape[1]
dim

768

**Step 2 : Build a FAISS Index for vectors**
    
    IndexFlatL2: this index use L 2 (Euclidean Distance) to create index that allows to do faster search

In [11]:
import faiss

index = faiss.IndexFlatL2(dim)

**Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index**



In [12]:
index.add(vectors) # adding vectors to that created index

In [13]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000251008C1510> >

**Step 4 : Encode search text using same encorder and normalize the output vector**



In [14]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query) # Converting above search query into vector
vec.shape

(768,)

In [15]:
import numpy as np
svec = np.array(vec).reshape(1,-1) # making vector two dimension array for index.search function format
svec.shape

(1, 768)

In [16]:
# faiss.normalize_L2(svec)

**Step 5: Search for similar vector in the FAISS index created**



In [22]:
distances, I = index.search(svec, k=2)
distances  # it returns tuple where first one is distance and scond one is index

array([[1.3844835, 1.4039094]], dtype=float32)

In [23]:
I #and scond one is index

array([[3, 2]], dtype=int64)

In [24]:
I.tolist()

[[3, 2]]

In [25]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [26]:
df.loc[row_indices] 

# with 3,2 index  it gives  below articles which are similar to our passed sentence which is related to Fashion also:  "I want to buy a polo t-shirt"

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [27]:
search_query

'I want to buy a polo t-shirt'

**You can see that the two results from the dataframe are similar to a search_query**



**so above method shows that it is semantic search since search_query exact words are not present in index 3,2 but both have same
category or context**